In [1]:
import numpy as np # primary math library
import matplotlib.pyplot as plt # primary plotting library
import numpy as np
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
%matplotlib qt
#%matplotlib notebook

# %matplotlib inline

import nelpy as nel # should be installed using pip install nelpy

import warnings
#warnings.filterwarnings("ignore") # useful to prevent excess nelpy warnings

In [2]:
dataroot = '/Users/ckemere/Development/Data/Frank/';

In [3]:
import os

filename = os.path.join(dataroot,'FrankLabData.nel')
import pandas as pd
import nelpy.io
jar = nel.load_pkl(filename)


In [4]:
data = jar.data
# if you are interested in the global task structure or characteristics of all the neurons 
#     recorded, they can be found in Pandas dataframes which are also saved in the file
taskinfo = jar.taskinfo
cellinfo = jar.cellinfo
tetinfo = jar.tetinfo

In [5]:
import numpy as np
def dot_auv(a,u,v):
    c = v[0]-u[0], v[1]-u[1]
    cnorm = np.sqrt(c[0]*c[0] + c[1]*c[1])
    dot = (c[1]*(a[1]-u[1]) + c[0]*(a[0]-u[0])) / cnorm
    return dot

def proj_auv(u,v,t):
    c = v[0]-u[0], v[1]-u[1]
    cnorm = np.sqrt(c[0]*c[0] + c[1]*c[1])
    return t*c[0]/cnorm + u[0],t*c[1]/cnorm+u[1]

class PathInteractor(object):
    """
    An path editor.

    Key-bindings

      't' toggle vertex markers on and off.  When vertex markers are on,
          you can move them, delete them


    """

    showverts = True
    epsilon = 5  # max pixel distance to count as a vertex hit

    def __init__(self, pathpatch):

        self.ax = pathpatch.axes
        canvas = self.ax.figure.canvas
        self.pathpatch = pathpatch
        self.pathpatch.set_animated(True)

        x, y = zip(*self.pathpatch.get_path().vertices)

        self.line, = ax.plot(x, y, marker='o', color='none',
                             markerfacecolor='r', animated=True)

        self._ind = None  # the active vert

        canvas.mpl_connect('draw_event', self.draw_callback)
        canvas.mpl_connect('button_press_event', self.button_press_callback)
        canvas.mpl_connect('key_press_event', self.key_press_callback)
        canvas.mpl_connect('button_release_event', self.button_release_callback)
        canvas.mpl_connect('motion_notify_event', self.motion_notify_callback)
        self.canvas = canvas

    def draw_callback(self, event):
        self.background = self.canvas.copy_from_bbox(self.ax.bbox)
        self.ax.draw_artist(self.pathpatch)
        self.ax.draw_artist(self.line)
        self.canvas.blit(self.ax.bbox)

    def pathpatch_changed(self, pathpatch):
        'this method is called whenever the pathpatchgon object is called'
        # only copy the artist props to the line (except visibility)
        vis = self.line.get_visible()
        plt.Artist.update_from(self.line, pathpatch)
        self.line.set_visible(vis)  # don't use the pathpatch visibility state

    def get_ind_under_point(self, event):
        'get the index of the vertex under point if within epsilon tolerance'

        # display coords
        xy = np.asarray(self.pathpatch.get_path().vertices)
        xyt = self.pathpatch.get_transform().transform(xy)
        xt, yt = xyt[:, 0], xyt[:, 1]
        d = np.sqrt((xt - event.x)**2 + (yt - event.y)**2)
        ind = d.argmin()

        if d[ind] >= self.epsilon:
            ind = None

        return ind

    def button_press_callback(self, event):
        'whenever a mouse button is pressed'
        if not self.showverts:
            return
        if event.inaxes is None:
            return
        if event.button != 1:
            return
        self._ind = self.get_ind_under_point(event)

    def button_release_callback(self, event):
        'whenever a mouse button is released'
        if not self.showverts:
            return
        if event.button != 1:
            return
        self._ind = None

    def key_press_callback(self, event):
        'whenever a key is pressed'
        if not event.inaxes:
            return
        if event.key == 't':
            self.showverts = not self.showverts
            self.line.set_visible(self.showverts)
            if not self.showverts:
                self._ind = None

        self.canvas.draw()

    def motion_notify_callback(self, event):
        'on mouse movement'
        if not self.showverts:
            return
        if self._ind is None:
            return
        if event.inaxes is None:
            return
        if event.button != 1:
            return
        x, y = event.xdata, event.ydata

        vertices = self.pathpatch.get_path().vertices

        vertices[self._ind] = x, y
        
        #if (self._ind == 0) | (self._ind ==2) :
            # vertices[1] = (vertices[0][0]+vertices[2][0])/2, (vertices[0][1]+vertices[2][1])/2
        if (self._ind == 1) :
            scaling = dot_auv(vertices[1], vertices[0],vertices[2])
            vertices[1] = proj_auv(vertices[0],vertices[2], scaling)
            vertices[4] = proj_auv(vertices[5],vertices[3], scaling)
        elif (self._ind == 4) :
            scaling = dot_auv(vertices[4], vertices[3],vertices[5])
            vertices[1] = proj_auv(vertices[0],vertices[2], scaling)
            vertices[4] = proj_auv(vertices[5],vertices[3], scaling)
        else:
            scaling = dot_auv(vertices[1], vertices[0],vertices[2])
            vertices[1] = proj_auv(vertices[0],vertices[2], scaling)
            scaling = dot_auv(vertices[4], vertices[5],vertices[3])
            vertices[4] = proj_auv(vertices[5],vertices[3], scaling)


        #if (self._ind == 3) | (self._ind ==5) :
            # vertices[4] = (vertices[3][0]+vertices[5][0])/2, (vertices[3][1]+vertices[5][1])/2


        self.line.set_data(zip(*vertices))

        self.canvas.restore_region(self.background)
        self.ax.draw_artist(self.pathpatch)
        self.ax.draw_artist(self.line)
        self.canvas.blit(self.ax.bbox)




In [9]:
from matplotlib.path import Path

x0 = 50
dx = 30

vertices = [
    (x0, x0),
    (x0+dx, x0),
    (x0+2*dx, x0),
    (x0+2*dx, x0+2*dx),
    (x0+dx, x0+2*dx),
    (x0, x0+2*dx),
    (x0, x0),
]

codes = [Path.MOVETO,
         Path.LINETO,
         Path.LINETO,
         Path.LINETO,
         Path.LINETO,
         Path.LINETO,
         Path.CLOSEPOLY,
         ]

path = Path(vertices, codes)

color_list = ['r','g','b','y','m','c','lightblue','lightgreen','lightred','brown','salmon','pink']
fig, ax = plt.subplots()
anim = 'Bon'
for idx, pp in data[anim].items():
    pos = pp['pos']
    # Novel
    #epochs = taskinfo.query('Animal==@anim & type=="run" & environment=="TrackB" & Day==@idx').Epoch.tolist()
    #if not epochs:
    #    continue
    # Familiar
    epochs = taskinfo.query('Animal==@anim & type=="run" & environment=="TrackA" & Day==@idx').Epoch.tolist()

    if (anim == 'Fra') & (idx == 3):
        epochs = [e-1 for e in epochs]
        
    tvals, vals = pos[epochs,:].asarray() # Access timestamps and channel data as numpy arrays
    xvals, yvals = vals # Access two channels as x and y
    ax.plot(xvals, yvals, lw=0.5, c='0.3')
    #ax.plot(xvals, yvals, lw=0.5, c=color_list.pop())


patch = mpatches.PathPatch(path, facecolor='none', edgecolor='blue', linewidth=3, alpha=0.5)
ax.add_patch(patch)

interactor = PathInteractor(patch)
plt.show()




/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of support
/Users/ckemere/Development/Python/nelpy/nelpy/core/_analogsignalarray.py:449: UserWarning: ignoring signal outside of 

In [7]:
NovelCenterBounds = {}
FamiliarCenterBounds = {}

In [10]:
FamiliarCenterBounds['Bon'] = path.vertices

In [11]:
FrankLabVertexData = nel.ResultsContainer(FamiliarCenterBounds=FamiliarCenterBounds,
                                          NovelCenterBounds=NovelCenterBounds,
                                    description='Data from the lab of Loren Frank CRCNS archive.\n')

In [12]:
FrankLabVertexData.save_pkl('FrankLabCenterArm.nel')